In [1]:
# === Import Libraries ===
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

# === Load Data ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

# === Drop Unused Columns ===
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

# === Extract Titles from Names ===
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# === Map Titles to Integers ===
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

# === Drop Name & PassengerId ===
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

# === Convert Sex to Numeric ===
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female': 1, 'male': 0}).astype(int)

# === Fill Missing Age Using RandomForest Regressor ===
for dataset in combine:
    age_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']
    known_age = dataset.dropna(subset=['Age'])
    unknown_age = dataset[dataset['Age'].isnull()]
    if unknown_age[age_features].isnull().values.any():
        age_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
        ])
        age_pipeline.fit(known_age[age_features], known_age['Age'])
        predicted_ages = age_pipeline.predict(unknown_age[age_features])
        dataset.loc[dataset['Age'].isnull(), 'Age'] = predicted_ages
    dataset['Age'] = dataset['Age'].astype(float).round().astype('Int64')

# === Bin Age into Categories ===
for dataset in combine:
    dataset['Age'] = pd.cut(dataset['Age'], bins=[0, 16, 32, 48, 64, np.inf],
                            labels=[0, 1, 2, 3, 4], include_lowest=True).astype(float).round().astype('Int64')

# === Create FamilySize and IsAlone ===
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# === Drop SibSp, Parch, FamilySize ===
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

# === Feature: Age*Class ===
for dataset in combine:
    dataset['Age*Class'] = dataset['Age'] * dataset['Pclass']

# === Fill Embarked ===
freq_port = train_df['Embarked'].dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

# === Fill missing Fare and Bin into Quartiles ===
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
for dataset in combine:
    dataset['Fare'] = pd.qcut(dataset['Fare'], 4, labels=[0, 1, 2, 3]).astype(int)

# === Prepare for Training ===
X_train = train_df.drop('Survived', axis=1)
Y_train = train_df['Survived']
X_test = test_df.drop('PassengerId', axis=1).copy()

# === Impute any leftover missing values (just in case) ===
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# === Train and Score Multiple Models ===
models = []

logreg = LogisticRegression()
logreg.fit(X_train_imputed, Y_train)
models.append(('Logistic Regression', round(logreg.score(X_train_imputed, Y_train) * 100, 2)))

svc = SVC(gamma='auto')
svc.fit(X_train_imputed, Y_train)
models.append(('Support Vector Machines', round(svc.score(X_train_imputed, Y_train) * 100, 2)))

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_imputed, Y_train)
models.append(('KNN', round(knn.score(X_train_imputed, Y_train) * 100, 2)))

gaussian = GaussianNB()
gaussian.fit(X_train_imputed, Y_train)
models.append(('Naive Bayes', round(gaussian.score(X_train_imputed, Y_train) * 100, 2)))

perceptron = Perceptron()
perceptron.fit(X_train_imputed, Y_train)
models.append(('Perceptron', round(perceptron.score(X_train_imputed, Y_train) * 100, 2)))

linear_svc = LinearSVC()
linear_svc.fit(X_train_imputed, Y_train)
models.append(('Linear SVC', round(linear_svc.score(X_train_imputed, Y_train) * 100, 2)))

sgd = SGDClassifier()
sgd.fit(X_train_imputed, Y_train)
models.append(('Stochastic Gradient Decent', round(sgd.score(X_train_imputed, Y_train) * 100, 2)))

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_imputed, Y_train)
models.append(('Decision Tree', round(decision_tree.score(X_train_imputed, Y_train) * 100, 2)))

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train_imputed, Y_train)
models.append(('Random Forest', round(random_forest.score(X_train_imputed, Y_train) * 100, 2)))

# === Show Model Comparison Table ===
models_df = pd.DataFrame(models, columns=['Model', 'Score'])
models_sorted = models_df.sort_values(by='Score', ascending=False)
display(models_sorted.style.format({'Score': '{:.2f}'}))

C:\Users\jayso\AppData\Local\Temp\ipykernel_6488\1708540394.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)


,Model,Score
7,Decision Tree,88.22
8,Random Forest,88.22
2,KNN,85.52
1,Support Vector Machines,83.73
0,Logistic Regression,81.37
5,Linear SVC,80.13
3,Naive Bayes,76.99
6,Stochastic Gradient Decent,66.44
4,Perceptron,64.87
